In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
text_input = 'an illustration of enjoying farming game ' #@param {type: "string"}

num_images_to_generate =  5#@param {type: "number"}

In [6]:
!pip install torch==1.9.1 torchtext==0.10.1 torchvision==0.10.1 torchaudio==0.10.1
%cd /content
!git clone https://github.com/drboog/Lafite
%cd /content/Lafite
!pip install git+https://github.com/openai/CLIP.git
!pip install ninja

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached torch-1.9.1-cp37-cp37m-manylinux1_x86_64.whl (831.4 MB)
  Using cached torchtext-0.10.1-cp37-cp37m-manylinux1_x86_64.whl (7.6 MB)
  Using cached torchvision-0.10.1-cp37-cp37m-manylinux1_x86_64.whl (22.1 MB)
  Using cached torchaudio-0.10.1-cp37-cp37m-manylinux1_x86_64.whl (2.9 MB)
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of torchtext to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of <Python from Requires-Python> to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
ER

In [7]:
!wget -N https://public-ipfs-gateway.pollinations.ai/ipfs/Qmdt4rch9AHveSh9JohEHqUGFMdfThsXJknwHjtQUQJQhe/pre-trained-google-cc-best-fid.pkl

--2022-05-25 06:43:26--  https://public-ipfs-gateway.pollinations.ai/ipfs/Qmdt4rch9AHveSh9JohEHqUGFMdfThsXJknwHjtQUQJQhe/pre-trained-google-cc-best-fid.pkl
Resolving public-ipfs-gateway.pollinations.ai (public-ipfs-gateway.pollinations.ai)... 65.108.44.19
Connecting to public-ipfs-gateway.pollinations.ai (public-ipfs-gateway.pollinations.ai)|65.108.44.19|:443... connected.
HTTP request sent, awaiting response... 502 Bad Gateway
2022-05-25 06:43:26 ERROR 502: Bad Gateway.



In [34]:
import torch
import numpy as np
import pickle
import os
import clip
from PIL import Image
from pathlib import Path
from tqdm import tqdm
import dnnlib, legacy
import clip
import torch.nn.functional as F
import torchvision.transforms as T
from tqdm import tqdm
import scipy

In [43]:
class Generator:
    def __init__(self, device, path):
        self.name = 'generator'
        self.model = self.load_model(device, path)
        self.device = device
        self.force_32 = False
        
    def load_model(self, device, path):
        with dnnlib.util.open_url(path) as f:
            network= legacy.load_network_pkl(f)
            self.G_ema = network['G_ema'].to(device)
            self.D = network['D'].to(device)
#                 self.G = network['G'].to(device)
            return self.G_ema
        
    def generate(self, z, c, fts, noise_mode='const', return_styles=True):
        return self.model(z, c, fts=fts, noise_mode=noise_mode, return_styles=return_styles, force_fp32=self.force_32)
    
    def generate_from_style(self, style, noise_mode='const'):
        ws = torch.randn(1, self.model.num_ws, 512)
        return self.model.synthesis(ws, fts=None, styles=style, noise_mode=noise_mode, force_fp32=self.force_32)
    
    def tensor_to_img(self, tensor):
        img = torch.clamp((tensor + 1.) * 127.5, 0., 255.)
        img_list = img.permute(0, 2, 3, 1)
        img_list = [img for img in img_list]
        return Image.fromarray(torch.cat(img_list, dim=-2).detach().cpu().numpy().astype(np.uint8))

In [44]:
from torch.nn import functional as F

In [45]:
with torch.no_grad():

    device = 'cuda:0' # please use GPU, do not use CPU
    path = '/content/drive/MyDrive/pre-trained-google-cc-best-fid.pkl'  # pre-trained model
    generator = Generator(device=device, path=path)
    clip_model, preprocess_clip  = clip.load("ViT-B/32", device=device)
    clip_model = clip_model.eval()
    
    tokenized_text = clip.tokenize([text_input]*num_images_to_generate).to(device)
    txt_fts = clip_model.encode_text(tokenized_text)
    txt_fts = txt_fts/txt_fts.norm(dim=-1, keepdim=True)
    
    z = torch.randn((num_images_to_generate, 512)).to(device)
    c = torch.randn((num_images_to_generate, 1)).to(device) # label is actually not used
    img, _ = generator.generate(z=z, c=c, fts=txt_fts)
    to_show_img = generator.tensor_to_img(img)

    to_show_img.save('./generated.jpg')


In [51]:
# text_input = an illustration of enjoying farming game 
tokenized_text=clip.tokenize([text_input]).to(device)
txt_ft = clip_model.encode_text(tokenized_text)
# txt_ft = txt_ft/txt_ft.norm(dim=-1, keepdim=True)
pross = preprocess_clip(to_show_img).to(device)
print(pross.shape)
print(img.shape)
print(txt_ft.shape)

torch.Size([3, 224, 224])
torch.Size([5, 3, 256, 256])
torch.Size([1, 512])


In [52]:
# txt_fts = clip_model.encode_text(tokenized_text)
scores = F.cosine_similarity(pross, txt_ft[:,:224]).squeeze()

rank = torch.argsort(scores, descending=True).cpu().numpy()

In [53]:
scores = scores.cpu().detach().numpy()

In [54]:
np.max(scores)

0.8198017

In [50]:
scores

array([[-0.71186733, -0.70087045,  0.7045098 , -0.6944767 ,  0.653255  ,
        -0.6423932 , -0.6188922 , -0.54562646, -0.5359875 ,  0.54207516,
        -0.5448884 , -0.57506883,  0.5725583 , -0.5433937 ,  0.51922345,
        -0.52502626, -0.51932496, -0.548256  ,  0.5529318 ,  0.5859928 ,
         0.57914   , -0.59432214,  0.6175961 , -0.6043026 , -0.58361626,
        -0.5744364 , -0.54249406,  0.54668283,  0.5376285 , -0.5336431 ,
         0.5264375 , -0.506258  , -0.5004402 ,  0.51087606, -0.51507145,
        -0.46100664,  0.3192197 ,  0.2945058 ,  0.37873596, -0.3951518 ,
        -0.39308903,  0.33032876,  0.33403862,  0.39297098,  0.35307965,
         0.35952413,  0.39288002, -0.36048594,  0.3984878 ,  0.42073917,
         0.34443527,  0.30555022, -0.38018328, -0.44004524,  0.42667103,
        -0.44351783,  0.3979759 , -0.4095387 , -0.37369147,  0.36303625,
         0.4438058 , -0.45241374,  0.4946893 ,  0.49347216,  0.54532516,
        -0.5940318 , -0.6660218 ,  0.66808397,  0.6